In [1]:
%pylab
%matplotlib inline
%load_ext watermark
%watermark -v -m -p scipy,numpy,sklearn,tensorflow

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib
CPython 3.5.2
IPython 5.1.0

scipy 0.18.1
numpy 1.11.3
sklearn 0.18.1
tensorflow 0.12.0

compiler   : GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)
system     : Darwin
release    : 16.3.0
machine    : x86_64
processor  : i386
CPU cores  : 8
interpreter: 64bit


In [2]:
from sklearn.datasets import load_digits
digits = load_digits()
digits_data = digits.data / 16

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(digits_data, digits.target, test_size=0.1, random_state=1)

In [4]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(n_values=10)
y_train_enc = ohe.fit_transform(y_train.reshape(-1, 1)).toarray()
y_test_enc = ohe.fit_transform(y_test.reshape(-1, 1)).toarray()
print(y_train_enc.shape, y_test_enc.shape)

(1617, 10) (180, 10)


In [5]:
import tensorflow as tf

In [6]:
X = tf.placeholder(tf.float32, [None, 64])
X_input = tf.reshape(X, [-1, 8, 8, 1])
Y = tf.placeholder(tf.float32, [None, 10])

W_1 = tf.Variable(tf.random_normal([3, 3, 1, 32], mean=0, stddev=1/np.sqrt(64)))
b_1 = tf.Variable(tf.random_normal([32], mean=0, stddev=1/np.sqrt(64)))
z_1 = tf.nn.conv2d(X_input, W_1, strides=[1, 1, 1, 1], padding='SAME') + b_1
h_1 = tf.nn.relu(z_1)

p_1 = tf.nn.max_pool(h_1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
p_1_flat = tf.reshape(p_1, [-1, 4*4*32])

W_2 = tf.Variable(tf.random_normal([4*4*32, 300], mean=0, stddev=1/np.sqrt(4*4*32)))
b_2 = tf.Variable(tf.random_normal([300], mean=0, stddev=1/np.sqrt(4*4*32)))
z_2 = tf.matmul(p_1_flat, W_2) + b_2
h_2 = tf.nn.relu(z_2)

W_3 = tf.Variable(tf.random_normal([300, 10], mean=0, stddev=1/np.sqrt(300)))
b_3 = tf.Variable(tf.random_normal([10], mean=0, stddev=1/np.sqrt(300)))
z_3 = tf.matmul(h_2, W_3) + b_3
y_ = tf.nn.softmax(z_3)

init = tf.global_variables_initializer()

cost_function = -tf.reduce_sum(Y * tf.log(y_))
optimizer = tf.train.GradientDescentOptimizer(0.0001).minimize(cost_function)

correct_prediction = tf.equal(tf.argmax(y_, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [7]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(2000):
        _, cost = sess.run([optimizer, cost_function], feed_dict={X: X_train, Y: y_train_enc})
        if i % 100 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={X: X_train, Y: y_train_enc})
            print("step %d, training accuracy %g, %g"%(i, cost/X_train.shape[0], train_accuracy))

    test_accuracy = sess.run(accuracy, feed_dict={X: X_test, Y: y_test_enc})
    print("test accuracy %g" % (test_accuracy))

step 0, training accuracy 2.29004, 0.168213
step 100, training accuracy 0.248097, 0.93321
step 200, training accuracy 0.0983939, 0.979592
step 300, training accuracy 0.0630571, 0.98825
step 400, training accuracy 0.0428284, 0.993816
step 500, training accuracy 0.03126, 0.996908
step 600, training accuracy 0.0237844, 0.996908
step 700, training accuracy 0.018652, 0.996908
step 800, training accuracy 0.0150017, 0.997526
step 900, training accuracy 0.0123227, 0.998763
step 1000, training accuracy 0.0103063, 0.999382
step 1100, training accuracy 0.00876254, 1
step 1200, training accuracy 0.00755651, 1
step 1300, training accuracy 0.00659935, 1
step 1400, training accuracy 0.00582309, 1
step 1500, training accuracy 0.00518464, 1
step 1600, training accuracy 0.00465386, 1
step 1700, training accuracy 0.00420816, 1
step 1800, training accuracy 0.00382953, 1
step 1900, training accuracy 0.00350546, 1
test accuracy 0.988889
